In [9]:
import numpy as np

from tensorflow import keras

from scikeras.wrappers import KerasClassifier

In [10]:
from sklearn.datasets import make_classification
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

In [18]:
N = 123
T = (N,)
print(type(T))

<class 'tuple'>


In [21]:
from typing import Dict, Iterable, Any

class MLPClassifier(KerasClassifier):

    def __init__(
        self,
        hidden_layer_sizes=100,
        activation="relu",
        batch_normalization=True,
        dropout=0.0,
        optimizer="adam",
        optimizer__learning_rate=0.001,
        epochs=200,
        verbose=0,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.batch_normalization = batch_normalization
        self.dropout = dropout
        self.optimizer = optimizer
        self.epochs = epochs
        self.verbose = verbose

    def _keras_build_fn(self, compile_kwargs: Dict[str, Any]):
        model = keras.Sequential()
        inp = keras.layers.Input(shape=(self.n_features_in_))
        model.add(inp)
        for hidden_layer_size in (self.hidden_layer_sizes,):
            layer = keras.layers.Dense(hidden_layer_size, activation=self.activation)
            model.add(layer)
            if self.batch_normalization:
               layer = keras.layers.BatchNormalization()
               model.add(layer)
            if self.dropout > 0.0:
               layer = keras.layers.Dropout(self.dropout)
               model.add(layer)    
        if self.target_type_ == "binary":
            n_output_units = 1
            output_activation = "sigmoid"
            loss = "binary_crossentropy"
        elif self.target_type_ == "multiclass":
            n_output_units = self.n_classes_
            output_activation = "softmax"
            loss = "sparse_categorical_crossentropy"
        else:
            raise NotImplementedError(f"Unsupported task type: {self.target_type_}")
        out = keras.layers.Dense(n_output_units, activation=output_activation)
        model.add(out)
        model.compile(loss=loss, optimizer=compile_kwargs["optimizer"])
        return model



In [22]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=20)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=5, min_lr=0.001)

clf = MLPClassifier(hidden_layer_sizes=20, activation="selu", dropout=0.3, metrics=['accuracy'], batch_size=64, epochs=2000, callbacks=[es, reduce_lr], validation_split=0.1, verbose=1)  # for notebook execution time

# check score
clf.fit(X, y)
print(clf.score(X, y))



Epoch 1/2000
15/15 [==============================] - 1s 7ms/step - loss: 0.8322 - val_loss: 0.9059 - lr: 0.0010
Epoch 2/2000
15/15 [==============================] - 0s 3ms/step - loss: 0.7749 - val_loss: 0.7729 - lr: 0.0010
Epoch 3/2000
15/15 [==============================] - 0s 2ms/step - loss: 0.7456 - val_loss: 0.6945 - lr: 0.0010
Epoch 4/2000
15/15 [==============================] - 0s 3ms/step - loss: 0.6887 - val_loss: 0.6421 - lr: 0.0010
Epoch 5/2000
15/15 [==============================] - 0s 2ms/step - loss: 0.6597 - val_loss: 0.6041 - lr: 0.0010
Epoch 6/2000
15/15 [==============================] - 0s 2ms/step - loss: 0.6655 - val_loss: 0.5771 - lr: 0.0010
Epoch 7/2000
15/15 [==============================] - 0s 3ms/step - loss: 0.6472 - val_loss: 0.5539 - lr: 0.0010
Epoch 8/2000
15/15 [==============================] - 0s 2ms/step - loss: 0.6260 - val_loss: 0.5439 - lr: 0.0010
Epoch 9/2000
15/15 [==============================] - 0s 2ms/step - loss: 0.5900 - val_loss: 0.5

In [23]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=500, n_features=5, noise=1, random_state=42)

In [24]:
from scikeras.wrappers import KerasRegressor

class MLPRegressor(KerasRegressor):

    def __init__(
        self,
        hidden_layer_sizes=100,
        activation="relu",
        batch_normalization=True,
        dropout=0,
        optimizer="adam",
        optimizer__learning_rate=0.001,
        epochs=200,
        verbose=0,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.batch_normalization = batch_normalization
        self.dropout = dropout
        self.optimizer = optimizer
        self.epochs = epochs
        self.verbose = verbose

    def _keras_build_fn(self, compile_kwargs: Dict[str, Any]):
        model = keras.Sequential()
        inp = keras.layers.Input(shape=(self.n_features_in_))
        model.add(inp)
        for hidden_layer_size in (self.hidden_layer_sizes,):
            layer = keras.layers.Dense(hidden_layer_size, activation=self.activation)
            model.add(layer)
            if self.batch_normalization:
               layer = keras.layers.BatchNormalization()
               model.add(layer)
            if self.dropout > 0:
               layer = keras.layers.Dropout(self.dropout)
               model.add(layer)    
        out = keras.layers.Dense(1)
        model.add(out)
        model.compile(loss="mse", optimizer=compile_kwargs["optimizer"])
        return model

In [25]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=20)
reduce_lr =keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=5, min_lr=0.001)

reg = MLPRegressor(hidden_layer_sizes=50, activation="selu", batch_normalization=True, dropout=0.5, batch_size=64, epochs=2000, callbacks=[es, reduce_lr], validation_split=0.1, verbose=1)  # for notebook execution time

# check score
print(reg.fit(X, y).score(X, y))

Epoch 1/2000
8/8 [==============================] - 0s 13ms/step - loss: 13606.5547 - val_loss: 8948.4385 - lr: 0.0010
Epoch 2/2000
8/8 [==============================] - 0s 4ms/step - loss: 13506.3857 - val_loss: 8918.2266 - lr: 0.0010
Epoch 3/2000
8/8 [==============================] - 0s 4ms/step - loss: 13445.3301 - val_loss: 8886.7402 - lr: 0.0010
Epoch 4/2000
8/8 [==============================] - 0s 4ms/step - loss: 13341.6611 - val_loss: 8853.5820 - lr: 0.0010
Epoch 5/2000
8/8 [==============================] - 0s 4ms/step - loss: 13245.8203 - val_loss: 8818.0771 - lr: 0.0010
Epoch 6/2000
8/8 [==============================] - 0s 4ms/step - loss: 13170.6963 - val_loss: 8779.9609 - lr: 0.0010
Epoch 7/2000
8/8 [==============================] - 0s 4ms/step - loss: 13114.1641 - val_loss: 8740.5811 - lr: 0.0010
Epoch 8/2000
8/8 [==============================] - 0s 4ms/step - loss: 13016.8809 - val_loss: 8699.1289 - lr: 0.0010
Epoch 9/2000
8/8 [==============================] - 0s 